In [1]:
pip install ollama


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


RAG Application using Ollama and Langchain

In [6]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_ollama import ChatOllama

In [2]:
raw_documents = TextLoader("./LangchainRetrieval.txt").load()

In [1]:
from langchain.document_loaders import PyPDFLoader

# Load PDF files
loader_1 = PyPDFLoader("./Herbs that synergize with TaxanesKadcylaEnhertu v7.pdf")
loader_2 = PyPDFLoader("./Herbs that synergize with Carboplatin and Cisplatin V3.pdf")
loader_3 = PyPDFLoader("./McKinney Surgery Pre and Post Op protocol.pdf")

# Load the documents
raw_documents_1 = loader_1.load()
raw_documents_2 = loader_2.load()
raw_documents_3 = loader_3.load()

# Combine all documents into one list
all_raw_documents = raw_documents_1 + raw_documents_2 + raw_documents_3


Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 21 0 (offset 0)
Ignoring wrong pointing object 31 0 (offset 0)
Ignoring wrong pointing object 37 0 (offset 0)
Ignoring wrong pointing object 39 0 (offset 0)
Ignoring wrong pointing object 41 0 (offset 0)
Ignoring wrong pointing object 43 0 (offset 0)
Ignoring wrong pointing object 45 0 (offset 0)
Ignoring wrong pointing object 59 0 (offset 0)
Ignoring wrong pointing object 61 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)


In [3]:
raw_documents= all_raw_documents

In [4]:
raw_documents

[Document(metadata={'producer': 'macOS Version 10.15.7 (Build 19H2026) Quartz PDFContext', 'creator': 'Word', 'creationdate': "D:20230827223908Z00'00'", 'title': 'Microsoft Word - Herbs that synergize with TaxanesKadcylaEnhertu v7.docx', 'moddate': "D:20230827223908Z00'00'", 'source': './Herbs that synergize with TaxanesKadcylaEnhertu v7.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='Herbs and Natural Substances Synergies with Taxanes  (Taxol, Paclitaxel, Docetaxel, Taxotere), Vinorelbine, Enhertu, and Kadcyla  \n Disclaimer: This document is for informational purposes, does not constitute medical advice, and should not replace the advice of a medical professional. \nSynergistic\tSubstances\twith\tChemotherapy\tSafe\tto\tuse\tthe\tfollowing\tduring\tchemotherapy\tcycle\tto\tincrease\teffectiveness,\tsupport\timmune\tsystem,\tand\treduce\tside\teffects:\tØ Curcumin\t(Start\tat\ta\tlow\tdose\tand\twork\tup.\tNo\teffect\ton\tCYP3A4\tin\thuman\tstudies)\tØ Medicinal\t

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=20)
documents = text_splitter.split_documents(raw_documents)

In [8]:
len(documents)

108

In [9]:
print(documents[0])
print(documents[1])

page_content='Herbs and Natural Substances Synergies with Taxanes  (Taxol, Paclitaxel, Docetaxel, Taxotere), Vinorelbine, Enhertu, and Kadcyla  
 Disclaimer: This document is for informational purposes, does not constitute medical advice, and should not replace the advice of a medical professional.' metadata={'producer': 'macOS Version 10.15.7 (Build 19H2026) Quartz PDFContext', 'creator': 'Word', 'creationdate': "D:20230827223908Z00'00'", 'title': 'Microsoft Word - Herbs that synergize with TaxanesKadcylaEnhertu v7.docx', 'moddate': "D:20230827223908Z00'00'", 'source': './Herbs that synergize with TaxanesKadcylaEnhertu v7.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}
page_content='Synergistic	Substances	with	Chemotherapy	Safe	to	use	the	following	during	chemotherapy	cycle	to	increase	effectiveness,	support	immune	system,	and	reduce	side	effects:	Ø Curcumin	(Start	at	a	low	dose	and	work	up.	No	effect	on	CYP3A4	in	human	studies)	Ø' metadata={'producer': 'macOS Version 10.15.7 (B

In [10]:
from langchain_ollama import OllamaEmbeddings

In [11]:
oembed = OllamaEmbeddings(base_url="http://localhost:11434", model="nomic-embed-text")

In [12]:
db = Chroma.from_documents(documents, embedding=oembed)

In [13]:
query = "Summarize the article about Carboplatin usage in cancer treatment."
docs = db.similarity_search(query)

In [14]:
len(docs)

4

In [15]:
print(docs[3].page_content)

inc,	and	selenium	can	blunt	tumor	resistance	to	cisplatin	and	carboplatin,	while	increasing	efficacy.	Milk	thistle	is	also	a	great	liver	protectant.	Theanine	increases	drug	concentration	in	a	tumor	through	inhibition	of	the	glutamate	transporter	via	the	GS-X	pump.	Omega	3	fish	oil	may	prevent	kidney


In [16]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [17]:
template = """Answer the question based only on the following context:

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [18]:
model = ChatOllama(
    model="llama3.1:latest",
    temperature=0
)

In [19]:
retriever = db.as_retriever()

In [20]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

In [21]:
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [22]:
chain.invoke("Summarize the article about Carboplatin usage in cancer treatment.")

'Here is a summary of the article about Carboplatin usage in cancer treatment:\n\nCarboplatin is a chemotherapy medication used to treat various types of cancer. While it is less toxic to the kidneys compared to Cisplatin, it has more side effects such as:\n\n* Electrolyte imbalances\n* Nausea and vomiting\n* Abnormal liver function\n* Nerve damage\n* Muscle pain\n\nTo mitigate these side effects, supplements like glutamine and selenium can be used. Additionally, certain herbs may enhance the effectiveness of Carboplatin or reduce tumor resistance to it, including:\n\n* Milk thistle (liver protectant)\n* Theanine (increases drug concentration in tumors)\n* Omega 3 fish oil (may prevent kidney damage)\n\nIt is essential to note that this article does not constitute medical advice and should not replace the guidance of a healthcare professional.'

In [23]:
chain.invoke("https://pmc.ncbi.nlm.nih.gov/articles/PMC10046228/")

"There is no question provided in the context you gave me. The text appears to be a collection of references and summaries related to cancer treatment and herbal interactions, but there is no actual question being asked. If you'd like to provide a specific question based on this context, I'll do my best to help!"